<a href="https://colab.research.google.com/github/Ayfred/MissionR-D/blob/main/SAFE_application2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
# name = "onlyEnComments && GooglePlayDescEN"
path = "/content/drive/MyDrive/Data/onlyEnComments.csv"
df_content = pd.read_csv(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Chargement des données CSV

In [ ]:
import csv

# open the CSV file using the csv module
with open(path, newline='',  encoding='utf-8') as csvfile:
    # create a csv reader object
    reader = csv.reader(csvfile)
    # create an empty list to store the strings
    string_list = []
    # loop through each row in the CSV file
    for row in reader:
        # add the string to the list
        #if detect(row[0]) == 'fr':
        string_list.append(row[0])

In [ ]:
#import jsonl & fichier annotations
import jsonlines
import json


path = "/content/drive/MyDrive/Data/AnnotedComments.jsonl"
annotations =[]


with jsonlines.open(path) as reader:
  i = 0
  for obj in reader:
    annotation = [i]
    for note in obj['entities']:
      annotation.append([note['start_offset'], note['end_offset']])
      print(annotation)
    if obj['entities']:
      annotations.append(annotation)
    i += 1



[0, [25, 39]]
[0, [25, 39], [43, 58]]
[0, [25, 39], [43, 58], [71, 75]]
[0, [25, 39], [43, 58], [71, 75], [80, 97]]
[0, [25, 39], [43, 58], [71, 75], [80, 97], [160, 170]]
[0, [25, 39], [43, 58], [71, 75], [80, 97], [160, 170], [176, 189]]
[0, [25, 39], [43, 58], [71, 75], [80, 97], [160, 170], [176, 189], [297, 318]]
[1, [43, 49]]
[1, [43, 49], [53, 64]]
[2, [19, 50]]
[3, [100, 121]]
[3, [100, 121], [131, 141]]
[3, [100, 121], [131, 141], [237, 252]]
[3, [100, 121], [131, 141], [237, 252], [260, 268]]


In [ ]:
annotations

[[0,
  [25, 39],
  [43, 58],
  [71, 75],
  [80, 97],
  [160, 170],
  [176, 189],
  [297, 318]],
 [1, [43, 49], [53, 64]],
 [2, [19, 50]],
 [3, [100, 121], [131, 141], [237, 252], [260, 268]]]

In [ ]:
string_list = string_list[:1000]

In [ ]:
#(string_list)

Pre-traitement du texte

In [ ]:
import re
import spacy

!python -m spacy download en_core_web_sm

2023-04-19 20:04:56.074364: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 54.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
#Etapes du texte Preprocessing
nlp = spacy.load("en_core_web_sm")
listStopWords = [str(x) for x in nlp.Defaults.stop_words]

NameError: ignored

In [ ]:
doc = nlp(string_list[0])
for token in doc:
  print(f"{token.text}")

Nicely
designed
,
has
a
good
selection
of
workout
options
,
and
does
a
good
job
tracking
my
sleep
(
via
a
Galaxy
Watch
4
)
.
I
do
wish
that
there
was
an
option
to
add
a
note
or
a
journal
entry
for
a
sleep
session
,
even
if
it
was
something
simple
.
In
regards
to
the
watch
app
,
I
think
there
should
be
more
breathing
cycles
in
the
"
Stress
"
section
(
you
can
only
add
up
to
30
cycles
/
7
mins
)
.


In [ ]:
def united(text):
    doc =  text.split(" ");
    # Itérer à travers chaque token dans le document
    for token in doc:
        if token in ["because", "since", "as", ]:
            doc = doc[:doc.index(token)]
            break
        #Remove explanations
        # Si le token est "parce que", "car" ou "puisque", supprimer tous les tokens suivants dans la phrase
    
    
    # Supprimer tout ce qui est entre parenthèses
    sentence = ' '.join(doc)
    #print(f"{sentence}")
    sentence = re.sub(r'\([^()]*\)', '', sentence)

    #print(f"remove explanations : {sentence}")

    sentence = re.sub('http[s]?://\S+', '', sentence)

    #print(f"remove links: {sentence}")


    # Regular expressions for phone numbers and email addresses
    quotes = r'\"[^\"]+\"'
    phone_regex = r"(?<!\d)(?:\d[ -/\\_\d]*){10}(?!\d)"
    email_regex = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"  # matches john.doe@example.com

    # Find phone numbers and email addresses in the sentence
    sentence = re.sub(quotes,"", sentence)

    #print(f"remove links2: {sentence}")


    sentence = re.sub(phone_regex,"", sentence)
    sentence = re.sub(email_regex,"", sentence)
    sentence = re.sub(r'[^\w]', ' ', sentence)
    doc = sentence.split(" ")
    for word in doc:
        if word in listStopWords:
            doc.remove(word)
    
    sentence = ' '.join(doc)

    #print(f"remove phone numbers and contacts: {sentence}")

    #Removal of ' and the char that precede it
    sentence = sentence.replace("'", " ")

    #print(f"remove leopaul: {sentence}")

    words = sentence.split(" ")
    newSentence = ""
    for word in words:
        if len(word) > 1:
            newSentence += word + " "
    sentence = newSentence
    
    #print(f"remove quotes : {sentence}")

    doc = nlp(sentence)
    #texte = doc.text
    text = ""
    for token in doc:
        if (token.pos_ == "PRON" or token.pos_ == "DET"):
            continue
        else:
            if not token.text.__contains__("'"):
                text += token.text
                text += " "
    sentence = text

    #print(f"remove pronoum and determinant : {sentence}")


    doc = nlp(sentence)
    result = []
    for token in doc:
        if not (token.ent_type_ == "ORG" or token.ent_type_ == "PERSON" or token.text.istitle()):
            result.append(token.text)
    sentence =  " ".join(result)
    #print(f"remove proper noun : {sentence}")

    return sentence

"""
for string in string_list:
    united_string = united(string)
    string_list_clean.append(united_string)
print(string_list_clean)
#run
#print(united(text))"""
united_string = []
for i in range(len(string_list)):
  united_string += [united(string_list[i])]



In [ ]:
#united_string

In [ ]:
annotation

[1999]

Apply SAFE POS pattern

In [ ]:
def superpose(da, fa, db, fb):
  return (da <= db <= fa | da <= fb <= fa | (db < da & fa < fb))


def matchTokens(id, start, end):
  both = 0
  for labeled in annotations:
    if id == labeled[0]:
      for i in range(1, len(labeled)):
        print(labeled[i][0], labeled[i][1], start, end)
        if superpose(labeled[i][0], labeled[i][1], start, end):
          both +=1
          break
  return both



In [ ]:
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher1 = Matcher(nlp.vocab)
matcher2 = Matcher(nlp.vocab)
matcher3 = Matcher(nlp.vocab)

"""def deleteTokens(matcher, doc, i, matches):
    # Get the current match and create tuple of entity label, start and end.
    # Append entity to the doc's entity. (Don't overwrite doc.ents!)
    match_id, start, end = matches[i]
    string_id = nlp.vocab.strings[match_id]
    text = ""

    nlp_list = list(doc)
    for i in range(end-start):
        text += nlp_list[start].text + " "
        del nlp_list[start]
    print("\n")
    print(text)
    print("\n")
    print(nlp_list)
    doc = nlp(" ".join([e.text for e in nlp_list]))
    matched_list.append(text)
    matched_id_list.append(string_id)"""


# Add match ID "HelloWorld" with no callback and one pattern
pattern1 = [{"POS": "NOUN"}, {"POS": "NOUN"}]
pattern2 = [{"POS": "VERB"}, {"POS": "NOUN"}]
pattern3 = [{"POS": "ADJ"}, {"POS": "NOUN"}]
pattern4 = [{"POS": "NOUN"}, {"POS": "CCONJ"}, {"POS": "NOUN"}]
pattern5 = [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
pattern6 = [{"POS": "NOUN"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
pattern7 = [{"POS": "VERB"}, {"POS": "PRON"}, {"POS": "NOUN"}]
pattern8 = [{"POS": "VERB"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
pattern9 = [{"POS": "VERB"}, {"POS": "ADJ"}, {"POS": "NOUN"}]
pattern10 = [{"POS": "ADJ"}, {"POS": "ADJ"}, {"POS": "NOUN"}]
pattern11 = [{"POS": "NOUN"}, {"POS": "ADP"}, {"POS": "NOUN"}]
pattern12 = [{"POS": "VERB"}, {"POS": "DET"}, {"POS": "NOUN"}]
pattern13 = [{"POS": "VERB"}, {"POS": "NOUN"}, {"POS": "ADP"}, {"POS": "NOUN"}]
pattern14 = [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
pattern15 = [{"POS": "ADJ"}, {"POS": "CONJ"}, {"POS": "ADJ"}]
pattern16 = [{"POS": "VERB"}, {"POS": "ADP"}, {"POS": "ADJ"}, {"POS": "NOUN"}]
pattern17 = [{"POS": "VERB"}, {"POS": "PRON"}, {"POS": "ADJ"}, {"POS": "NOUN"}]
pattern18 = [{"POS": "NOUN"}, {"POS": "CCONJ"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
matcher1.add("18", [pattern18])
matcher1.add("17", [pattern17])
matcher1.add("16", [pattern16])
matcher2.add("15", [pattern15])
matcher1.add("14", [pattern14])
matcher1.add("13", [pattern13])
matcher2.add("12", [pattern12])
matcher2.add("11", [pattern11])
matcher2.add("10", [pattern10])
matcher2.add("9", [pattern9])
matcher2.add("8", [pattern8])
matcher2.add("7", [pattern7])
matcher2.add("6", [pattern6])
matcher2.add("5", [pattern5])
matcher2.add("4", [pattern4])
matcher3.add("3", [pattern3])
matcher3.add("2", [pattern2])
matcher3.add("1", [pattern1])

matched_id_list = []
matched_list = []
#doc = nlp("Song and artist album,  Create your greatest album, ")

for row in range(len(united_string)):
  doc = nlp(united_string[row])
  while(True):
    matches = matcher1(doc)
    for match_id, start, end in matches:
      trupos = matchTokens(row, start, end)
      string_id = nlp.vocab.strings[match_id]
      nlp_list = list(doc)
      #print(doc[start:end])
      matched_list.append(doc[start:end])
      matched_id_list.append(string_id)

      for i in range(end-start):
        if(start<len(nlp_list)):
          del nlp_list[start]

      doc = nlp(" ".join([e.text for e in nlp_list]))
    if (not matches):
      break
  while(True):
    matches = matcher1(doc)
    #trupos += matchTokens(row, matches)
    for match_id, start, end in matches:
      string_id = nlp.vocab.strings[match_id]
      nlp_list = list(doc)
      print(doc[start:end])
      matched_list.append(doc[start:end])
      matched_id_list.append(string_id)

      for i in range(end-start):
        if(start<len(nlp_list)):
          del nlp_list[start]

      doc = nlp(" ".join([e.text for e in nlp_list]))
    if (not matches):
      break
  while(True):
    matches = matcher1(doc)
    #trupos += matchTokens(row, matches)
    for match_id, start, end in matches:
      string_id = nlp.vocab.strings[match_id]
      nlp_list = list(doc)
      print(doc[start:end])
      matched_list.append(doc[start:end])
      matched_id_list.append(string_id)

      for i in range(end-start):
        if(start<len(nlp_list)):
          del nlp_list[start]

      doc = nlp(" ".join([e.text for e in nlp_list]))
    if (not matches):
      break

print(trupos)
#Remove duplicate and synonyms, noise
print(matched_list)

25 39 1 5
43 58 1 5
71 75 1 5
80 97 1 5
160 170 1 5
176 189 1 5
297 318 1 5
25 39 6 10
43 58 6 10
71 75 6 10
80 97 6 10
160 170 6 10
176 189 6 10
297 318 6 10
25 39 26 30
43 58 26 30
71 75 26 30
80 97 26 30
160 170 26 30
176 189 26 30
297 318 26 30
25 39 2 6
43 58 2 6
71 75 2 6
80 97 2 6
160 170 2 6
176 189 2 6
297 318 2 6
25 39 18 22
43 58 18 22
71 75 18 22
80 97 18 22
160 170 18 22
176 189 18 22
297 318 18 22
25 39 14 18
43 58 14 18
71 75 14 18
80 97 14 18
160 170 14 18
176 189 14 18
297 318 14 18
0
[good selection workout options, wish option add note, , good job tracking sleep, , more breathing cycles section, accurate jnside apartment building, building, walking halls near parameter, little ways instance scheduling, , good meassure steps distance, enter data back sleep, best feature fact review, route feature, disappointing app return map, longer workout notifications steps, past week use galaxy, won load calendar longer, showing on main screen, turn phone for minute, let sign wit

In [ ]:
for el in matched_list:
  if len(el.text.split(" "))<=1:
    matched_list.remove(el)

In [ ]:
len(matched_list)

In [ ]:
print(matched_list)

In [ ]:
matched_list

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
!pip install faiss-cpu
import faiss
from sklearn.metrics import silhouette_score
import numpy as np
from sentence_transformers import SentenceTransformer

# Load pre-trained language model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define number of clusters
num_clusters = 63

# Define the list of phrases to cluster
phrases = matched_list

# Encode the phrases using the pre-trained model
phrase_embeddings = model.encode(phrases)

# Cluster the embeddings using K-means
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(phrase_embeddings)

# Compute the silhouette score for the clustering
silhouette_avg = silhouette_score(phrase_embeddings, kmeans.labels_)

print("The average silhouette_score is :", silhouette_avg)

# Assign each phrase to a cluster
clusters = [[] for i in range(num_clusters)]
for i, label in enumerate(kmeans.labels_):
    clusters[label].append(phrases[i])

# Print the clusters
for i in range(num_clusters):
    print("Cluster {}:".format(i+1))
    print(clusters[i])


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import numpy as np
import faiss

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = matched_list
corpus_embeddings = embedder.encode(corpus)

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Set up Faiss for nearest neighbor search
index = faiss.IndexFlatIP(corpus_embeddings.shape[1])
index.add(corpus_embeddings)

# Perform nearest neighbor search to get indices of similar embeddings
_, indices = index.search(corpus_embeddings, k=5)  # Get the 5 most similar embeddings for each sentence

# Perform DBSCAN clustering on the similar embeddings
clustering_model = DBSCAN(eps=0.42, min_samples=5, metric='cosine')

# Perform DBSCAN clustering on the similar embeddings
clustering_model = DBSCAN(eps=0.42, min_samples=5, metric='euclidean')

# Perform HDBSCAN clustering on the similar embeddings
clustering_model = hdbscan.HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_epsilon=0.40)


cluster_assignment = clustering_model.fit_predict(corpus_embeddings[indices[:, 1]])

# Compute the silhouette score for the clustering
silhouette_avg = silhouette_score(corpus_embeddings[indices[:, 1]], cluster_assignment)

print("The average silhouette_score is :", silhouette_avg)

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(cluster)
    print("")


In [ ]:
"""
This is a simple application for sentence embeddings: clustering
Sentences are mapped to sentence embeddings and then k-mean clustering is applied.
"""
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

embedder = SentenceTransformer('all-mpnet-base-v2')

# Corpus with example sentences
corpus = matched_list
corpus_embeddings = embedder.encode(corpus)

# Perform kmean clustering
num_clusters = 63
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

In [ ]:
"""
This is a simple application for sentence embeddings: clustering

Sentences are mapped to sentence embeddings and then agglomerative clustering with a threshold is applied.
"""
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = matched_list
corpus_embeddings = embedder.encode(corpus)

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

# Compute the silhouette score for the clustering
silhouette_avg = silhouette_score(corpus_embeddings, cluster_assignment)

print("The average silhouette_score is :", silhouette_avg)

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(cluster)
    print("")

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import numpy as np

embedder = SentenceTransformer('all-mpnet-base-v2')

# Corpus with example sentences
corpus = matched_list
corpus_embeddings = embedder.encode(corpus)

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Perform DBSCAN clustering
clustering_model = DBSCAN(eps=0.5, min_samples=2, metric='cosine')
cluster_assignment = clustering_model.fit_predict(corpus_embeddings)

# Compute the silhouette score for the clustering
silhouette_avg = silhouette_score(corpus_embeddings[indices[:, 1]], cluster_assignment)

print(silhouette_avg)

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(cluster)
    print("")


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import numpy as np
import faiss
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = matched_list
corpus_embeddings = embedder.encode(corpus)

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Set up Faiss for nearest neighbor search
index = faiss.IndexFlatIP(corpus_embeddings.shape[1])
index.add(corpus_embeddings)

# Perform nearest neighbor search to get indices of similar embeddings
_, indices = index.search(corpus_embeddings, k=5)  # Get the 5 most similar embeddings for each sentence

# Perform DBSCAN clustering on the similar embeddings
clustering_model = DBSCAN(eps=1.5, min_samples=2, metric='cosine')
cluster_assignment = clustering_model.fit_predict(corpus_embeddings[indices[:, 1]])


# Reduce dimensionality for visualization
pca = PCA(n_components=2)
corpus_embeddings_2d = pca.fit_transform(corpus_embeddings)

# Plot the clustered embeddings
fig, ax = plt.subplots()
scatter = ax.scatter(corpus_embeddings_2d[:, 0], corpus_embeddings_2d[:, 1], c=cluster_assignment)
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper right", title="Clusters")
ax.add_artist(legend1)
plt.show()


In [ ]:
!pip install hdbscan

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import hdbscan
from sklearn.metrics import silhouette_score

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = matched_list
corpus_embeddings = embedder.encode(corpus)

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Set up Faiss for nearest neighbor search
index = faiss.IndexFlatIP(corpus_embeddings.shape[1])
index.add(corpus_embeddings)

# Perform nearest neighbor search to get indices of similar embeddings
_, indices = index.search(corpus_embeddings, k=5)  # Get the 5 most similar embeddings for each sentence

# Perform HDBSCAN clustering on the similar embeddings
clustering_model = hdbscan.HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_epsilon=0.40)
cluster_assignment = clustering_model.fit_predict(corpus_embeddings[indices[:, 1]])

# Compute the silhouette score for the clustering
silhouette_avg = silhouette_score(corpus_embeddings[indices[:, 1]], cluster_assignment)

print("The average silhouette_score is :", silhouette_avg)

from sklearn.metrics import calinski_harabasz_score

ch_score = calinski_harabasz_score(corpus_embeddings[indices[:, 1]], cluster_assignment)

print("The Calinski-Harabasz score is :", ch_score)

from sklearn.metrics import davies_bouldin_score

db_score = davies_bouldin_score(corpus_embeddings[indices[:, 1]], cluster_assignment)

print("The Davies-Bouldin score is :", db_score)



clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(cluster)
    print("")


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import networkx as nx

# List of features
features = matched_list

# Preprocess the features
# You can use NLTK or spaCy for preprocessing
preprocessed_features = [feat.lower().split() for feat in features]

# Create a document-term matrix
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform([' '.join(feat) for feat in preprocessed_features])

# Identify topics using LDA
lda = LatentDirichletAllocation(n_components=3, random_state=0)
doc_topic_dist = lda.fit_transform(dtm)

# Create a topic graph
G = nx.Graph()
for i, feat in enumerate(features):
    topic = doc_topic_dist[i].argmax()
    G.add_node(feat, topic=topic)

# Add edges between nodes with the same topic
for u, v in G.edges():
    if G.nodes[u]['topic'] == G.nodes[v]['topic']:
        G.add_edge(u, v)

# Draw the topic graph
pos = nx.spring_layout(G, k=0.5, iterations=50)
colors = ['red', 'green', 'blue']
for topic in range(3):
    nodes = [node for node in G.nodes() if G.nodes[node]['topic'] == topic]
    nx.draw_networkx_nodes(G, pos, nodelist=nodes, node_color=colors[topic], alpha=0.8)
nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.5)
nx.draw_networkx_labels(G, pos, font_size=10, font_family='sans-serif')
plt.axis('off')
plt.show()


In [ ]:
!pip install gensim

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import numpy as np
import faiss
#from gensim.summarization import keywords

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = matched_list
corpus_embeddings = embedder.encode(corpus)

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Set up Faiss for nearest neighbor search
index = faiss.IndexFlatIP(corpus_embeddings.shape[1])
index.add(corpus_embeddings)

# Perform nearest neighbor search to get indices of similar embeddings
_, indices = index.search(corpus_embeddings, k=5)  # Get the 5 most similar embeddings for each sentence

# Perform DBSCAN clustering on the similar embeddings
clustering_model = DBSCAN(eps=1.5, min_samples=2, metric='euclidean')


cluster_assignment = clustering_model.fit_predict(corpus_embeddings[indices[:, 1]])

# Automatically extract keywords from sentences in each cluster and use them as cluster names
clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

cluster_names = {}
for cluster_id, sentences in clustered_sentences.items():
    keywords_str = keywords('\n'.join(sentences))
    cluster_names[cluster_id] = keywords_str.split('\n')[0] if keywords_str else f"Cluster {cluster_id+1}"

# Print clusters with their corresponding names
for cluster_id, sentences in clustered_sentences.items():
    print(f"Cluster {cluster_id+1} - {cluster_names[cluster_id]}")
    print(sentences)
    print("")


In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = matched_list[1:]
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = [matched_list[0]]

for queries in matched_list:
  # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
  top_k = min(100, len(corpus))
  for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    for score, idx in zip(top_results[0], top_results[1]):
        if(score > 0,6):
          matched_list.pop(idx)
      


In [ ]:
!pip install faiss-cpu

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, util

#On charge le modèle de transformation de phrases "all-MiniLM-L6-v2" à partir de la bibliothèque SentenceTransformer.

model = SentenceTransformer('all-MiniLM-L6-v2')

#le nombre de voisins les plus proches à récupérer.
# Define the number of nearest neighbors to retrieve
k = 5

# Define the corpus of sentences
corpus = matched_list[1:]

# Encoder le corpus de phrases en utilisant le modèle de transformation de phrases.
corpus_embeddings = model.encode(corpus)

# Embed the query sentence
query = matched_list[0]
query_embedding = model.encode([query])

# Create an index for the corpus embeddings
#On crée un index FAISS pour les embeddings des phrases dans le corpus,
#en utilisant l'algorithme "IndexFlatIP" pour calculer le produit scalaire entre les embeddings.
d = len(corpus_embeddings[0])
index = faiss.IndexFlatIP(d)
index.add(np.array(corpus_embeddings))

# Search for the k nearest neighbors
D, I = index.search(np.array(query_embedding), k)

# Print the results
print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 5 most similar sentences in corpus:")
for i, idx in enumerate(I[0]):
    print(corpus[idx], "(Score: {:.4f})".format(D[0][i]))

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import DBSCAN

#On charge le modèle de transformation de phrases "all-MiniLM-L6-v2" à partir de la bibliothèque SentenceTransformer.

model = SentenceTransformer('all-MiniLM-L6-v2')

#le nombre de voisins les plus proches à récupérer.
# Define the number of nearest neighbors to retrieve
k = 5

# Define the corpus of sentences
corpus = matched_list[1:]

# Encoder le corpus de phrases en utilisant le modèle de transformation de phrases.
corpus_embeddings = model.encode(corpus)

# Embed the query sentence
query = matched_list[0]
query_embedding = model.encode([query])

# Create an index for the corpus embeddings
#On crée un index FAISS pour les embeddings des phrases dans le corpus,
#en utilisant l'algorithme "IndexFlatIP" pour calculer le produit scalaire entre les embeddings.
d = len(corpus_embeddings[0])
index = faiss.IndexFlatIP(d)
index.add(np.array(corpus_embeddings))

# Search for the k nearest neighbors
D, I = index.search(np.array(query_embedding), k)

# Get the features to be clustered
features = np.array(matched_list)

# Define the parameters for the DBSCAN clustering algorithm
eps = 0.5
min_samples = 5

# Instantiate a DBSCAN object with the defined parameters
dbscan = DBSCAN(eps=eps, min_samples=min_samples)

# Fit the DBSCAN object to the features
dbscan.fit(features)

# Get the labels for each point in the features
labels = dbscan.labels_

# Get the unique labels (i.e., cluster IDs) in the labels array
unique_labels = np.unique(labels)

# Loop through each label to print the corresponding features
for label in unique_labels:
    if label == -1:
        # This is noise, i.e., a feature that doesn't belong to any cluster
        print("Noise:")
    else:
        # This is a cluster
        print("Cluster", label, ":")
    # Print the features in the current cluster
    for idx, feature in enumerate(features[labels == label]):
        print("\t", idx+1, ":", feature)

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import hdbscan
from sklearn.metrics import silhouette_score

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = matched_list
corpus_embeddings = embedder.encode(corpus)

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Set up Faiss for nearest neighbor search
index = faiss.IndexFlatIP(corpus_embeddings.shape[1])
index.add(corpus_embeddings)

# Perform nearest neighbor search to get indices of similar embeddings
_, indices = index.search(corpus_embeddings, k=5)  # Get the 5 most similar embeddings for each sentence

# Perform HDBSCAN clustering on the similar embeddings
clustering_model = hdbscan.HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_epsilon=0.40)
cluster_assignment = clustering_model.fit_predict(corpus_embeddings[indices[:, 1]])

# Compute the silhouette score for the clustering
silhouette_avg = silhouette_score(corpus_embeddings[indices[:, 1]], cluster_assignment)

print("The average silhouette_score is :", silhouette_avg)

from sklearn.metrics import calinski_harabasz_score

ch_score = calinski_harabasz_score(corpus_embeddings[indices[:, 1]], cluster_assignment)

print("The Calinski-Harabasz score is :", ch_score)

from sklearn.metrics import davies_bouldin_score

db_score = davies_bouldin_score(corpus_embeddings[indices[:, 1]], cluster_assignment)

print("The Davies-Bouldin score is :", db_score)



clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(cluster)
    print("")


In [ ]:
matched_list_str = [str(k) for k in matched_list]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import networkx as nx

# List of features
features = matched_list_str # Your list of 1000+ features

# Preprocess the features
# You can use NLTK or spaCy for preprocessing
preprocessed_features = [feat.lower().split() for feat in features]

# Create a document-term matrix
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform([' '.join(feat) for feat in preprocessed_features])

# Identify topics using LDA
lda = LatentDirichletAllocation(n_components=30, random_state=0)
doc_topic_dist = lda.fit_transform(dtm)

# Create a topic graph
G = nx.Graph()
for i, feat in enumerate(features):
    topic = doc_topic_dist[i].argmax()
    G.add_node(feat, topic=topic)

# Add edges between nodes with the same topic
for u, v in G.edges():
    if G.nodes[u]['topic'] == G.nodes[v]['topic']:
        G.add_edge(u, v)

# Draw the topic graph
pos = nx.spring_layout(G, k=0.5, iterations=50)
colors = ['red', 'green', 'blue']
for topic in range(3):
    nodes = [node for node in G.nodes() if G.nodes[node]['topic'] == topic]
    nx.draw_networkx_nodes(G, pos, nodelist=nodes, node_color=colors[topic], alpha=0.8)
nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.5)
nx.draw_networkx_labels(G, pos, font_size=10, font_family='sans-serif')
plt.axis('off')
plt.figure(figsize=(20, 15))
plt.show()


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import networkx as nx

# List of features
features = matched_list_str[:100]  # Your list of 1000+ features

# Preprocess the features
# You can use NLTK or spaCy for preprocessing
preprocessed_features = [feat.lower().split() for feat in features]

# Create a document-term matrix
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform([' '.join(feat) for feat in preprocessed_features])

# Identify topics using LDA
n_topics = 1
lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)
doc_topic_dist = lda.fit_transform(dtm)

# Create a topic graph
G = nx.Graph()
for i, feat in enumerate(features):
    topic = doc_topic_dist[i].argmax()
    G.add_node(feat, topic=topic, prob=doc_topic_dist[i][topic])

# Add edges between nodes with the same topic
for u, v in G.edges():
    if G.nodes[u]['topic'] == G.nodes[v]['topic']:
        G.add_edge(u, v)

# Filter out features with low topic probability
top_n = 100
for topic in range(n_topics):
    nodes = [node for node in G.nodes() if G.nodes[node]['topic'] == topic]
    nodes_sorted = sorted(nodes, key=lambda x: G.nodes[x]['prob'], reverse=True)
    nodes_top = nodes_sorted[:top_n]
    for node in nodes_top:
        G.nodes[node]['label'] = node.split()[-1]

# Draw the topic graph
pos = nx.spring_layout(G, k=0.5, iterations=50)
colors = plt.cm.get_cmap('hsv', n_topics)
for topic in range(n_topics):
    nodes = [node for node in G.nodes() if G.nodes[node]['topic'] == topic]
    nodes_top = [node for node in nodes if 'label' in G.nodes[node]]
    node_colors = [colors(topic) for _ in range(len(nodes_top))]
    nx.draw_networkx_nodes(G, pos, nodelist=nodes_top, node_color=node_colors, alpha=0.8)
nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.5)
node_labels = nx.get_node_attributes(G, 'label')
nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=10, font_family='sans-serif')
plt.axis('off')
plt.figure(figsize=(30, 20))
plt.show()


In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, util

#On charge le modèle de transformation de phrases "all-MiniLM-L6-v2" à partir de la bibliothèque SentenceTransformer.

model = SentenceTransformer('all-MiniLM-L6-v2')

#le nombre de voisins les plus proches à récupérer.
# Define the number of nearest neighbors to retrieve
k = 5

# Define the corpus of sentences
corpus = matched_list[1:]

# Encoder le corpus de phrases en utilisant le modèle de transformation de phrases.
corpus_embeddings = model.encode(corpus)

# Embed the query sentence
query = matched_list[0]
query_embedding = model.encode([query])

# Create an index for the corpus embeddings
#On crée un index FAISS pour les embeddings des phrases dans le corpus,
#en utilisant l'algorithme "IndexFlatIP" pour calculer le produit scalaire entre les embeddings.
d = len(corpus_embeddings[0])
index = faiss.IndexFlatIP(d)
index.add(np.array(corpus_embeddings))

print(index)

# Search for the k nearest neighbors
D, I = index.search(np.array(query_embedding), k)

# Print the results
print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 5 most similar sentences in corpus:")
for i, idx in enumerate(I[0]):
    print(corpus[idx], "(Score: {:.4f})".format(D[0][i]))

In [ ]:
list1 = ["good job tracking sleep",
"enter data back sleep",
"total sleep time graph",
"overall sleep patterns note",
"able edit sleep times",
"actual time records sleep",
"simple sleep tracker fitness", 'reminder automatic sleep detection',       'automatic activity tracker sleep',       'non food tracking barcode',       'select day week selector',       'regular meals have transferred',       'meals plus custom meals',       'motivational speech minimum interruption',       'persistent notification app running',       'paper plus follow work',       'daily activity rings resetting',       'achieve day from team',       'educational articles nutrition exercise',       'daily food track nutrition',       'food instance doesn track',       'daily track steps exercise',       'intake day track calories',       'breastfeeding moms for pregnancy',       'new achiever level badge',        'daily basis heart rate']

In [ ]:
list2 =["good job tracking sleep", "total sleep time graph", "overall sleep patterns note", "own step feature edit", "skip login option watch", "actual step count ratio", "goals thanks perks far", "yearly challenge friends term", "yearly step km goal", "total steps miles kilometers", "yearly goals steps distance", "mandatory step verification glitches", "other sleep data sleep", "day sleep occurred night", "simple sleep tracker fitness", "figured sleep of turn", "total seconds month sleep", "morning and notice thinks", "exact route day app", "deep sleep stage hour", "choose sleep from notification", "real challenge ability export", "total duration speed tracker", "ran consecutive ever reaching", "decent step calorie counter", "more steps discrepancy app"]

In [ ]:
from sklearn import metrics
metrics.adjusted_rand_score(list1, list2)

In [ ]:
list1 = ["good job tracking sleep"]; list2 = [""]

In [ ]:
list1 = [1, 1, 0, 0, 3, 3]; list2 = [1, 2, 0, 1, 2, 3]

In [ ]:
set1 = set(list1)
set2 = set(list2)

intersection = set1.intersection(set2)
union = set1.union(set2)

similarity = len(intersection) / len(union)

print(f"Jaccard similarity: {similarity}")
